# Elastic Stackの構築手順：インストール
----
Elasticsearchをインストールし、 各種パラメータを設定してクラスタを構築します。  
必要であればこの後のNotebookで利用するサンプルデータも投入します。

## 設定情報の引き継ぎ

アカウントや通信先などを読み込みます。

In [1]:
%run rc.py
(USER, KEYPATH)

('centos', '~/.ssh/ansible_id_rsa')

In [2]:
%env ES_CLIENT={es_host}:9200

env: ES_CLIENT=34.213.xx.xx:9200


このNotebookを含むファイルを設置したディレクトリを読み込みます。  
※Logstash用の設定ファイルやサンプルデータはこの配下から取得します。変更する場合は下の設定を書き換えてください。

In [3]:
import os
notebooks_home = os.getcwd()
notebooks_home

'/notebooks/03_Elasticsearch\xe6\xa7\x8b\xe7\xaf\x89\xe6\x89\x8b\xe9\xa0\x86\xe3\x81\xb8\xe3\x81\xae\xe9\x81\xa9\xe7\x94\xa8\xe4\xbe\x8b'

## インストール

Elasticsearchのインストールし、設定、起動します。

### jdkのインストール
Elasticsearchを動作させるために、Javaをインストールします。

次のコマンドを実行してインストールします。

In [4]:
!ansible-playbook playbooks/install_jdk.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [i-0bf412a529d12xxxx]

TASK [install jdk] *************************************************************
changed: [i-0bf412a529d12xxxx]

TASK [set JAVA_HOME] ***********************************************************
changed: [i-0bf412a529d12xxxx]

TASK [set PATH] ****************************************************************
changed: [i-0bf412a529d12xxxx]

PLAY RECAP *********************************************************************
i-0bf412a529d12xxxx        : ok=4    changed=3    unreachable=0    failed=0   



### Elasticsearchのインストール
Elasticsearchのパッケージをインストールします。

次のコマンドを実行します。

In [5]:
!ansible-playbook playbooks/install_elasticsearch-5.0.0.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [i-0bf412a529d12xxxx]

TASK [install elasticsearch 5.0.0] *********************************************
changed: [i-0bf412a529d12xxxx]

PLAY RECAP *********************************************************************
i-0bf412a529d12xxxx        : ok=2    changed=1    unreachable=0    failed=0   



### Elasticsearchの設定
Elasticsearchの設定を変更します。以下の2つを反映します。
- [01_03_Set_Inventory.ipynb](01_03_Set_Inventory.ipynb)で設定したhosts（インベントリ）のグループ
- [01_01_Outline.ipynb](01_01_Outline.ipynb)で出力したgroup_vars/all（パラメータ）

次のコマンドを実行して、elasticsearch.ymlを設定します。

In [6]:
!ansible-playbook playbooks/set_elasticsearch.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [i-0bf412a529d12xxxx]

TASK [set_cluster-name] ********************************************************
changed: [i-0bf412a529d12xxxx]

TASK [set_network-host] ********************************************************
changed: [i-0bf412a529d12xxxx]

TASK [set_node-master] *********************************************************
changed: [i-0bf412a529d12xxxx]

TASK [set_node-data] ***********************************************************
changed: [i-0bf412a529d12xxxx]

TASK [set node-ingest] *********************************************************
changed: [i-0bf412a529d12xxxx]

TASK [set_discovery_zen_minimum_master_nodes] **********************************
changed: [i-0bf412a529d12xxxx]

TASK [set_discovery_zen_ping_unicast_hosts] ************************************
changed: [i-0bf412a529d12xxxx]

PLAY RECAP *******

次のコマンドを実行して、ヒープサイズを設定します。/etc/elasticsearch/ にあるjvm.optionsにJavaの起動オプションとして指定します。

In [7]:
!ansible-playbook playbooks/set_heap-size.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [i-0bf412a529d12xxxx]

TASK [set -Xms jvm option] *****************************************************
changed: [i-0bf412a529d12xxxx]

TASK [set -Xmx jvm option] *****************************************************
changed: [i-0bf412a529d12xxxx]

PLAY RECAP *********************************************************************
i-0bf412a529d12xxxx        : ok=3    changed=2    unreachable=0    failed=0   



次のコマンドを実行して、カーネルパラメータを変更します。  
/etc/sysctl.conf のvm.max_map_count に 262144 を設定しています。

In [8]:
!ansible-playbook playbooks/set_kernelparameter.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [i-0bf412a529d12xxxx]

TASK [Set vm.max_map_count to 262144 in /etc/sysctl.conf] **********************
changed: [i-0bf412a529d12xxxx]

PLAY RECAP *********************************************************************
i-0bf412a529d12xxxx        : ok=2    changed=1    unreachable=0    failed=0   



### Elasticsearchの起動
Elasticsearchを起動します。

次のコマンドを実行します。

In [9]:
!ansible-playbook playbooks/start_cluster.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [i-0bf412a529d12xxxx]

TASK [start elasticsearch] *****************************************************
changed: [i-0bf412a529d12xxxx]

PLAY RECAP *********************************************************************
i-0bf412a529d12xxxx        : ok=2    changed=1    unreachable=0    failed=0   



各サーバの起動は次のコマンドで確認してください。

In [10]:
!ansible all -m shell -b -a "systemctl status elasticsearch" -i ./hosts -u $USER --private-key=$KEYPATH

i-0bf412a529d12xxxx | SUCCESS | rc=0 >>
● elasticsearch.service - Elasticsearch
   Loaded: loaded (/usr/lib/systemd/system/elasticsearch.service; disabled; vendor preset: disabled)
   Active: active (running) since Sun 2017-07-30 12:53:43 UTC; 9s ago
     Docs: http://www.elastic.co
  Process: 11051 ExecStartPre=/usr/share/elasticsearch/bin/elasticsearch-systemd-pre-exec (code=exited, status=0/SUCCESS)
 Main PID: 11053 (java)
   CGroup: /system.slice/elasticsearch.service
           └─11053 /bin/java -Xms31g -Xmx31g -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=75 -XX:+UseCMSInitiatingOccupancyOnly -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -server -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Dlog4j.skipJansi=true -XX:+HeapDumpOnOutOfMemoryError -Des.path.home=/usr/share/elasticsearch -cp /usr/share/elasticsearch/lib/elasticsearch-5.

次のように  
**Active: active (running)**    
と表示されれば正常に起動しています。

以下のコマンドでクラスタのヘルスチェックをすることができます。

> 以下のセルを実行した際に、 `Failed to connect to 10.24.128.128 port 9200: Connection refused` などと表示される場合は、まだサービスが起動しきれていない可能性があります。しばらく待って、再度セル実行を試してみてください。

In [11]:
!curl -XGET http://$ES_CLIENT/_cluster/health?pretty

{
  "cluster_name" : "es-cluster",
  "status" : "green",
  "timed_out" : false,
  "number_of_nodes" : 1,
  "number_of_data_nodes" : 1,
  "active_primary_shards" : 0,
  "active_shards" : 0,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 0,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 100.0
}


### Elasticsearchのテンプレートの投入
Shard数、Replica数を、インデックステンプレートの形でで投入することで設定します。

[Index Templates](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-templates.html)とはインデックス単位の設定やマッピングを定義したもので、インデックス生成時に適用されます。  
投入はAPIにて実施します。よってElasticsearchが起動したあとの投入となります。

次のコマンドを実行して、テンプレートを投入します。

> 以下のコマンド実行に失敗する場合は、 [#Elasticsearchの起動](#Elasticsearchの起動) にてサービスが起動しているか、ヘルスチェックに応答しているかを確認するようにしてください。

In [12]:
!ansible-playbook playbooks/set_api_parameter.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [data-nodes[0]] ***********************************************************

TASK [Gathering Facts] *********************************************************
ok: [i-0bf412a529d12xxxx]

TASK [post param] **************************************************************
ok: [i-0bf412a529d12xxxx]

PLAY RECAP *********************************************************************
i-0bf412a529d12xxxx        : ok=2    changed=0    unreachable=0    failed=0   



投入した内容は次のコマンドで表示できます。

In [13]:
!curl -XGET $ES_CLIENT/_template/template_base?pretty

{
  "template_base" : {
    "order" : 0,
    "template" : "*",
    "settings" : {
      "index" : {
        "number_of_shards" : "1",
        "number_of_replicas" : "0"
      }
    },
    "mappings" : { },
    "aliases" : { }
  }
}


- テンプレートには任意の名称を指定できます。このコマンドではtemplate_baseという名称にしています。
- **"template"**パラメータは、このテンプレートを適用する先のインデックス名を指定しています。
  - **"template" : "*" **としているので全インデックスに適用されます。
- あるインデックスが複数のテンプレートにマッチする場合、**"order"** に指定した数が大きいものが優先されます。
  - このコマンドでは **"order" : 0 ** としているので最低の優先度となっており、よってどんな追加の指定でも上書き可能です。
- **"number_of_shards"**、**"number_of_replicas" **でShard数、Replica数を指定しています。


テンプレートの変更を投入すること自体は任意のタイミングで可能ですが、変更前に投入されたインデックスに変更結果は適用されません。  
インデックスの投入前に必要なテンプレートを投入する必要があります。  
詳細はElasticsearch Referenceの[Index Templates](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-templates.html)を参照してください。

### Logstashのインストール
Logstashのパッケージをインストールします。

次のコマンドを実行します。

In [14]:
!ansible-playbook playbooks/install_logstash-5.0.0.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [logstash-server] *********************************************************

TASK [Gathering Facts] *********************************************************
ok: [i-0bf412a529d12xxxx]

TASK [install logstash 5.0.0] **************************************************
changed: [i-0bf412a529d12xxxx]

PLAY RECAP *********************************************************************
i-0bf412a529d12xxxx        : ok=2    changed=1    unreachable=0    failed=0   



## クラスタのステータスの確認方法と監視方法
クラスタのステータスを確認するためにCluster APIを発行します。

> ※本節では、クラスタの簡単な状態を確認するAPIのみを紹介します。  
> 　詳細の確認やエラーの把握、各種リソースの状態チェックなどを実施したい場合は  
> 　**[01_05_Diagnostics.ipynb](01_05_Diagnostics.ipynb)**を利用してください。

概要を表示したい場合にはCluster Health APIを用います。  
クラスタの状態や、クラスタに属しているノードの数などを見ることができます。

In [15]:
!curl -XGET http://$ES_CLIENT/_cluster/health?pretty

{
  "cluster_name" : "es-cluster",
  "status" : "green",
  "timed_out" : false,
  "number_of_nodes" : 1,
  "number_of_data_nodes" : 1,
  "active_primary_shards" : 0,
  "active_shards" : 0,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 0,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 100.0
}


上記のコマンドを実行して得られる出力の中で、**status**や**number_of_nodes**を見れば  
クラスタの状態およびクラスタ内のノード数を確認することができます。  
データ投入直後などは、クラスタの状態がyellowになっている場合があります。その際はgreenになったことを確認してから検索などを行ってください。

出力例

より詳細な情報を確認したい場合にはCluster Stats APIを用います。

In [16]:
!curl -XGET http://$ES_CLIENT/_cluster/stats?pretty

{
  "_nodes" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "cluster_name" : "es-cluster",
  "timestamp" : 1501419291846,
  "status" : "green",
  "indices" : {
    "count" : 0,
    "shards" : { },
    "docs" : {
      "count" : 0,
      "deleted" : 0
    },
    "store" : {
      "size_in_bytes" : 0,
      "throttle_time_in_millis" : 0
    },
    "fielddata" : {
      "memory_size_in_bytes" : 0,
      "evictions" : 0
    },
    "query_cache" : {
      "memory_size_in_bytes" : 0,
      "total_count" : 0,
      "hit_count" : 0,
      "miss_count" : 0,
      "cache_size" : 0,
      "cache_count" : 0,
      "evictions" : 0
    },
    "completion" : {
      "size_in_bytes" : 0
    },
    "segments" : {
      "count" : 0,
      "memory_in_bytes" : 0,
      "terms_memory_in_bytes" : 0,
      "stored_fields_memory_in_bytes" : 0,
      "term_vectors_memory_in_bytes" : 0,
      "norms_memory_in_bytes" : 0,
      "points_memory_in_bytes" : 0,
      "doc_values_memory_in_bytes" 

このコマンドでは、シャードやメモリに関する情報を取得することができます。  
詳細はElasticsearch Referenceの[Cluster Stats](https://www.elastic.co/guide/en/elasticsearch/reference/current/cluster-stats.html)を参照してください。

### Monitoringによる監視

Monitoring([公式ページ](https://www.elastic.co/guide/en/x-pack/current/xpack-monitoring.html))はサブスクリプションを購入すると利用できる「X-Pack」をインストールすることで有効になる機能です。  

Elasticsearchの稼働状況を監視するプラグインで、CPU使用率やディスクサイズ、Latencyなどの情報を取得することができます。  
取得した情報はElasticsearchに登録されるので、あとから遡ってクラスタのステータスを確認することも可能です。  

また、X-Packに含まれるWatcherを用いることで、特定の条件下で通知を行うこともできます。  
例） CPU使用率が特定の値を超えた、ディスク空き容量が一定量を下回ったなど。

## Logstashへのサンプルデータ投入（オプション）
インストール済みのLogstashに、この後のNotebookで利用するサンプルデータを投入します。

＃**この後の「02_Search.ipynb」以降のサンプルコマンドを実施する場合は、本節によりサンプルデータを投入してください。**  
＃**サンプルデータが不要であれば、この節の内容はスキップしてください。**

### 環境設定ファイルの設置
Logstash用の環境設定ファイルをサーバにコピーします。

サンプルデータをElasticsearchに登録するための設定は、[logstash_conf/main.conf](logstash_conf/main.conf)を用います。  
LogstashサーバがData Nodeと同じホストに無い場合は、  
main.confの "**hosts => ～～**" の部分にClient Nodeか、それが無い場合はData Nodeのリストを定義してください。  
複数の場合は  
["127.0.0.1:9200","127.0.0.2:9200"]  
のようにします。

次のリンクから修正してください。

- **[main.conf](../edit/logstash_conf/main.conf)**

ここで利用するmain.confは標準入力で読み込んだCSV形式の内容をElasticsearchに格納するための設定です。  
具体的な意味は[04_Store_Data](04_Store_Data.ipynb)で説明します。

main.confを /etc/logstash/conf.d/ 配下にコピーします。

In [17]:
!ansible-playbook playbooks/copy_logstash_conf.yml --extra-vars "notebooks_home=$notebooks_home" -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [logstash-server] *********************************************************

TASK [Gathering Facts] *********************************************************
ok: [i-0bf412a529d12xxxx]

TASK [copy local main.conf to remote] ******************************************
changed: [i-0bf412a529d12xxxx]

PLAY RECAP *********************************************************************
i-0bf412a529d12xxxx        : ok=2    changed=1    unreachable=0    failed=0   



### データ型の定義（マッピング定義）
サンプルデータ用のCSVの各列について、データ型をあらかじめ定義しておきます。

データ型の種類およびマッピング定義の詳細に関しては[05_Indexing](05_Indexing.ipynb)を参照してください。  
次のコマンドを実行してマッピング定義を行います。  

In [18]:
%%bash
curl -XPUT "http://$ES_CLIENT/_template/weather" -d @- << EOF
{
  "template" : "meteorological-data-*",
  "mappings": {
    "logs": {
      "dynamic_templates" : [
        {
          "my_strings" : {
            "match_mapping_type" : "string",
            "mapping" : {
              "type" : "keyword"
            }
          }
        }
      ],
      "properties": {
        "atmospheric_pressure": {
          "type": "float"
        },
        "sea_level_pressure":{
          "type": "float"
        },
        "precipitation_day": {
          "type": "integer"
        },
        "precipitation_day": {
          "type": "float"
        },
        "precipitation_max_hour":{
          "type":"float"
        },
        "precipitation_max_10min":{
          "type":"float"
        },
        "temperature_avg":{
          "type":"float"
        },
        "temperature_max":{
          "type":"float"
        },
        "temperature_min":{
          "type":"float"
        },
        "humidity_avg":{
          "type":"float"
        },
        "humidity_min":{
          "type":"float"
        },
        "wind_speed_avg":{
          "type":"float"
        },
        "wind_speed_max":{
          "type":"float"
        },
        "wind_speed_max_moment":{
          "type":"float"
        },
        "sunshine_duration":{
          "type":"float"
        },
        "snowfall":{
          "type":"float"
        },
        "snowfall_max":{
          "type":"float"
        }
      }
    }
  }
}
EOF

{"acknowledged":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1394  100    21  100  1373     34   2274 --:--:-- --:--:-- --:--:--  2276


### サンプルデータの投入
実際のサンプルデータをLogstashに投入します。

ここでは気象庁から提供されている気象データを利用します。
[気象庁「過去の気象データ・ダウンロード」](http://www.data.jma.go.jp/gmd/risk/obsdl/)

公開されているデータのうち、2015年の東京のデータ(1年分)を本Notebookで利用するCSV形式に変換し、サンプルデータとして利用します([tokyo2015.csv](sample_data/tokyo2015.csv))。

サンプルデータと気象庁データの項目は、次のように対応しています。

| サンプルデータの項目 | 気象庁データの項目 | 備考 |
|-----------|--------------------|------|
| @timestamp | 年月日 ||
| location | 地点 | 気象庁データに直接の記載はないため、変換時に追加した |
| atmospheric_pressure | 日平均現地気圧 ||
| sea_level_pressure | 日平均海面気圧 ||
| precipitation_day | 降水量の日合計 ||
| precipitation_max_hour | 1時間降水量の日最大 ||
| precipitation_max_10min | 10分間降水量の日最大 ||
| temperature_avg | 日平均気温 ||
| temperature_max | 日最高気温 ||
| temperature_min | 日最低気温 ||
| humidity_avg | 日平均相対湿度 ||
| humidity_min | 日最小相対湿度 ||
| wind_speed_avg | 日平均風速 ||
| wind_speed_max | 日最大風速 ||
| wind_direction | 日最大風速（風向） ||
| wind_speed_max_moment | 日最大瞬間風速 ||
| wind_direction_max_moment | 日最大瞬間風速（風向） ||
| sunshine_duration | 日照時間 ||
| snowfall | 降雪量の日合計 ||
| snowfall_max | 日最深積雪 ||
| information_daytime | 天気概況（昼：06時～18時） ||
| information_night | 天気概況（夜：18時～翌日06時） |||

気象庁データの利用あたっては気象庁が公開している[利用規約](http://www.jma.go.jp/jma/kishou/info/coment.html)を参照してください。

登録するデータの内容は次のコマンドで確認できます。1行目がデータの項目を表し、2行目以降が実際のデータです。  
Elasticsearchは慣習として時刻情報を@timestampというフィールドに格納します。

In [19]:
!head -n 5 ./sample_data/tokyo2015.csv

@timestamp,location,atmospheric_pressure,sea_level_pressure,precipitation_day,precipitation_max_hour,precipitation_max_10min,temperature_avg,temperature_max,temperature_min,humidity_avg,humidity_min,wind_speed_avg,wind_speed_max,wind_direction,wind_speed_max_moment,wind_direction_max_moment,sunshine_duration,snowfall,snowfall_max,information_daytime,information_night
2015-01-01T00:00:00,tokyo,1000.4,1003.4,0,0,0,3.7,8.2,0.7,41.0,30.0,4.4,9.4,西,18.7,北西,1.3,,,時々晴一時雪,晴
2015-01-02T00:00:00,tokyo,1007.4,1010.4,,,,2.7,7.9,-2.2,41.0,19.0,2.1,4.2,西,7.8,西,6.0,,,時々曇,一時曇
2015-01-03T00:00:00,tokyo,1012.0,1015.1,,,,3.8,8.9,-1.1,42.0,19.0,3.0,7.4,北西,10.9,北西,8.9,,,晴,後薄曇
2015-01-04T00:00:00,tokyo,1010.4,1013.4,,,,4.0,9.3,-0.4,51.0,33.0,1.3,2.8,北西,3.8,北東,6.0,,,後晴,晴


Logstashのサーバーにサンプルデータをコピーします。   
Logstashをインストールしたサーバーの ~/ 配下に[sample_data](sample_data)のtokyo2015.csvをコピーします。

In [20]:
!ansible-playbook playbooks/copy_sample_data.yml --extra-vars "notebooks_home=$notebooks_home" -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [logstash-server] *********************************************************

TASK [Gathering Facts] *********************************************************
ok: [i-0bf412a529d12xxxx]

TASK [copy local sample data to remote] ****************************************
changed: [i-0bf412a529d12xxxx]

PLAY RECAP *********************************************************************
i-0bf412a529d12xxxx        : ok=2    changed=1    unreachable=0    failed=0   



Logstashを使ってサンプルデータをElasticsearchに登録します。次のセルのコマンドを実行してください。

In [21]:
!ansible logstash-server -m shell -a "cat tokyo2015.csv | sudo /usr/share/logstash/bin/logstash --path.settings /etc/logstash -f /etc/logstash/conf.d/main.conf" -i ./hosts -u $USER --private-key=$KEYPATH

i-0bf412a529d12xxxx | SUCCESS | rc=0 >>
Sending Logstash logs to /var/log/logstash which is now configured via log4j2.properties.



データが登録されているかどうかは、次のセルを実行することで確認できます。

In [22]:
!curl -XGET http://$ES_CLIENT/meteorological-data-*/_search/?pretty

{
  "took" : 34,
  "timed_out" : false,
  "_shards" : {
    "total" : 13,
    "successful" : 13,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.01",
        "_type" : "logs",
        "_id" : "tokyo_15-01-01",
        "_score" : 1.0,
        "_source" : {
          "date" : "15-01-01",
          "wind_speed_avg" : "4.4",
          "snowfall" : null,
          "temperature_max" : "8.2",
          "precipitation_day" : "0",
          "wind_speed_max" : "9.4",
          "sunshine_duration" : "1.3",
          "temperature_avg" : "3.7",
          "temperature_min" : "0.7",
          "@version" : "1",
          "wind_direction_max_moment" : "北西",
          "information_night" : "晴",
          "precipitation_max_10min" : "0",
          "precipitation_max_hour" : "0",
          "wind_direction" : "西",
          "information_daytime" : "時々晴一時雪",
          "@timestamp" : "2015-01-01T00:00:00.000Z",
 

登録したデータの内容がJSON形式で表示されるので、それを確認してください。  
レスポンスのサイズを指定していないため、ここでは最大10件まで確認できます。  

### 分散配置の状態確認
投入したでデータがData Nodeにどのように配置されているか確認します。

次のコマンドで、Data NodeごとにPrimary/Replica Shardが分散配置されている状況を確認できます。

In [23]:
!curl -XGET http://$ES_CLIENT/_cat/shards?v

index                       shard prirep state   docs  store ip            node
meteorological-data-2015.07 0     p      STARTED   31 26.9kb 10.30.xxx.xxx fGNTupV
meteorological-data-2015.05 0     p      STARTED   29 33.9kb 10.30.xxx.xxx fGNTupV
meteorological-data-2015.11 0     p      STARTED   28 24.9kb 10.30.xxx.xxx fGNTupV
meteorological-data-2015.01 0     p      STARTED   29 25.4kb 10.30.xxx.xxx fGNTupV
meteorological-data-2017.07 0     p      STARTED    0   130b 10.30.xxx.xxx fGNTupV
meteorological-data-2015.09 0     p      STARTED   30 36.8kb 10.30.xxx.xxx fGNTupV
meteorological-data-2015.02 0     p      STARTED   28 24.4kb 10.30.xxx.xxx fGNTupV
meteorological-data-2015.03 0     p      STARTED   31 26.6kb 10.30.xxx.xxx fGNTupV
meteorological-data-2015.12 0     p      STARTED   31 26.4kb 10.30.xxx.xxx fGNTupV
meteorological-data-2015.06 0     p      STARTED   29 26.1kb 10.30.xxx.xxx fGNTupV
meteorological-data-2015.04 0     p      STARTED   30 26.4kb 10.30.xxx.xxx fGNTupV
meteoro

例えばShard数が3、Replica数が1の場合、あるインデックスの分の表示は次のようになります。

左列からの順に次の意味です。

|列名|意味|値の例|
|----|----|----|
|index|インデックス名|meteorological-data-2015.09|
|shard|Shard番号|1|
|prirep|Primary(p)かReplica(r)か|p|
|state|状態|STARTED|
|docs|ドキュメント数|11|
|store|ディスクサイズ|31.4kb|
|ip|配置先のData NodeのIP|172.31.27.2|


※Shardの配置のイメージについては[01_01_Outline.ipynbの「Elasticsearchクラスタ」](01_01_Outline.ipynb#Elasticsearchクラスタ)の章にある図を参照してください。

よって、次のようになっていればShard数、Replica数は正しく反映されています。
- Shard番号の数の種類がShard数と同じだけある  
（例えばShard数が3であれば、Shard番号は0,1,2の3種類あるはず）
- あるインデックスのShard番号ごとに、"r"のついた行がReplica数だけある